<a href="https://colab.research.google.com/github/joker2017/Calculator/blob/master/29122018_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
print( os.listdir('/content') )

Saving imagenet_class_names.txt to imagenet_class_names (3).txt
User uploaded file "imagenet_class_names.txt" with length 31674 bytes
['.config', 'imagenet_class_names (1).txt', 'imagenet_class_names.txt', 'inception_v3_2016_08_28', 'imagenet_class_names (3).txt', 'flower_photos', 'imagenet_class_names (2).txt', 'sample_data']


In [3]:


#Загрузите последнюю предварительную модель Inception v3: контрольная точка доступна по адресу https://github.com/tensorflow/models/tree/master/research/slim . 
#писок имен классов доступен по адресу https://goo.gl/brXRtZ , но вы должны сначала вставить «background» класс.
import os
import re 
import sys
import tarfile
import numpy as np
import tensorflow as tf
from random import sample
from six.moves import urllib
import matplotlib.image as mpimg
import tensorflow.contrib.slim as slim
from tensorflow.contrib.slim.nets import inception
from skimage.transform import rescale, resize, downscale_local_mean

MODEL_NAME = "inception_v3.ckpt"
CLASS_NAME_REGEX = re.compile(r"^n\d+\s+(.*)\s*$", re.M | re.U)
DATASET_URL = "http://download.tensorflow.org/example_images/flower_photos.tgz"
MODEL_URL = "http://download.tensorflow.org/models/inception_v3_2016_08_28.tar.gz"



def download_progress(count, block_size, total_size):
    percent = count * block_size * 100 // total_size
    sys.stdout.write("\rDownloading: {}%".format(percent))
    sys.stdout.flush()
    

def download_tgz(url):
    file_name=os.path.basename(url)
    name_parts = os.path.split(file_name)
    _, name = name_parts
    name_path = name.rsplit( ".", 2 )[ 0 ]
    path = os.path.join("/content", name_path)
    tgz_path = os.path.join(path, name)
    if os.path.exists(path): #and os.path.isfile(tgz_path):
      print("file and path exist ", tgz_path, "Path is:", os.listdir(path), "!")
      return path
    else:
      print(os.path.exists(path), os.path.isfile(tgz_path), path, tgz_path)
      os.makedirs(path, exist_ok=True)
      urllib.request.urlretrieve(url, tgz_path, reporthook=download_progress)
      tmp_tgz = tarfile.open(tgz_path)
      tmp_tgz.extractall(path=path)
      tmp_tgz.close()
      os.remove(tgz_path)
      print("path created:", path)
      return path
    
# Загружаем модель
MODELS_PATH = download_tgz(MODEL_URL)
# Загружаем датасет
FLOWERS_PATH = download_tgz(DATASET_URL)
 

def load_class_names():
    with open(os.path.join("/content", "imagenet_class_names.txt"), "rb") as f:
        content = f.read().decode("utf-8")
        #sys.stdout.flush()
        return CLASS_NAME_REGEX.findall(content)
      
load_class_names()    
class_names = ["background"] + load_class_names()       


file and path exist  /content/inception_v3_2016_08_28/inception_v3_2016_08_28.tar.gz Path is: ['inception_v3.ckpt'] !
file and path exist  /content/flower_photos/flower_photos.tgz Path is: ['flower_photos'] !


In [0]:
from tensorflow.contrib.slim.nets import inception
import tensorflow.contrib.slim as slim

tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=[None, height, width, channels], name="X")
training = tf.placeholder_with_default(False, shape=[])
with slim.arg_scope(inception.inception_v3_arg_scope()):
    logits, end_points = inception.inception_v3(X, num_classes=1001, is_training=training)

inception_saver = tf.train.Saver()

end_points

In [0]:
flower_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="InceptionV3")
flower_vars

In [0]:
#MODEL_URL
import os
name_parts = os.path.splitext(MODEL_URL)  
file, name = name_parts
print(file, name)
name_parts = os.path.split(MODEL_URL)  
file, name = name_parts
print(file, name)
print(name.rsplit( ".", 2 )[ 0 ]) 
print(os.path.dirname(MODEL_URL))

In [0]:
import os
print( os.getcwd() )
print( os.listdir('/content/flower_photos/flower_photos') )

In [0]:
class_names[:5]

In [13]:
width = 299
height = 299
channels = 3
import tensorflow as tf
tf.reset_default_graph()
n_epochs = 1000
batch_size = 100
best_loss_val = np.infty
check_interval = 500
checks_since_last_progress = 0
max_checks_without_progress = 20
best_model_params = None 





#Каждый подкаталог flower_photos каталога содержит все изображения данного класса. Давайте получим список классов:
flowers_root_path = os.path.join(FLOWERS_PATH, "flower_photos")
flower_classes = sorted([dirname for dirname in os.listdir(flowers_root_path)
                  if os.path.isdir(os.path.join(flowers_root_path, dirname))])

#Давайте получим список всех путей файлов изображений для каждого класса:

from collections import defaultdict
image_paths = defaultdict(list)
for flower_class in flower_classes:
    image_dir = os.path.join(flowers_root_path, flower_class)
    for filepath in os.listdir(image_dir):
        if filepath.endswith(".jpg"):
            image_paths[flower_class].append(os.path.join(image_dir, filepath))
            
            
#Давайте сортируем пути изображения, чтобы заставить этот ноутбук вести себя последовательно на нескольких запусках:
for paths in image_paths.values():
    paths.sort() 

    
#Для получения дополнительных функций манипуляции с изображениями, таких как вращения, проверьте документацию SciPy или эту приятную страницу .
def prepare_image(image, target_width = 299, target_height = 299, max_zoom = 0.2):
    """Zooms and crops the image randomly for data augmentation."""

    
    # Во-первых, давайте найдем самый большой ограничивающий прямоугольник с целевым соотношением размеров, которое вписывается в изображение
    height = image.shape[0]
    width = image.shape[1]
    image_ratio = width / height
    target_image_ratio = target_width / target_height
    crop_vertically = image_ratio < target_image_ratio
    crop_width = width if crop_vertically else int(height * target_image_ratio)
    crop_height = int(width / target_image_ratio) if crop_vertically else height
        
   #Теперь давайте уменьшим этот ограничивающий прямоугольник случайным фактором (разделив размеры на случайное число
   # между 1.0 и 1.0 + `max_zoom`
    resize_factor = np.random.rand() * max_zoom + 1.0
    crop_width = int(crop_width / resize_factor)
    crop_height = int(crop_height / resize_factor)
    
    # Далее мы можем выбрать случайное место на изображении этого прямоугольника.
    x0 = np.random.randint(0, width - crop_width)
    y0 = np.random.randint(0, height - crop_height)
    x1 = x0 + crop_width
    y1 = y0 + crop_height
    
    # Давайте обрезать изображение, используя случайную ограничивающую рамку, которую мы построили.
    image = image[y0:y1, x0:x1]

    # Давайте также перевернем изображение по горизонтали с вероятностью 50% :
    if np.random.rand() < 0.5:
        image = np.fliplr(image)

    # Теперь давайте изменим размер изображения до целевых размеров.
    image = resize(image, (target_width, target_height), mode='reflect')
    
    # Наконец, давайте убедимся, что цвета представлены как
    # 32-бит плавает в диапазоне от 0.0 до 1.0 (на данный момент):
    return image.astype(np.float32) / 255
  

def prepare_batch(flower_paths_and_classes, batch_size):
    batch_paths_and_classes = sample(flower_paths_and_classes, batch_size)
    images = [mpimg.imread(path)[:, :, :channels] for path, labels in batch_paths_and_classes]
    prepared_images = [prepare_image(image) for image in images]
    X_batch = 2 * np.stack(prepared_images) - 1 # Inception expects colors ranging from -1 to 1
    y_batch = np.array([labels for path, labels in batch_paths_and_classes], dtype=np.int32)
    return X_batch, y_batch 
  
  
def prepare_data(flower_paths_and_classes):
    batch_paths_and_classes = flower_paths_and_classes
    images = [mpimg.imread(path)[:, :, :channels] for path, labels in batch_paths_and_classes]
    prepared_images = [prepare_image(image) for image in images]
    X_data = 2 * np.stack(prepared_images) - 1 # Inception expects colors ranging from -1 to 1
    y_data = np.array([labels for path, labels in batch_paths_and_classes], dtype=np.int32)
    return X_data, y_data  
  
  
#раннняя остановка
def get_model_params():
    gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    return {gvar.op.name: value for gvar, value in zip(gvars, tf.get_default_session().run(gvars))}

def restore_model_params(model_params):
    gvar_names = list(model_params.keys())
    assign_ops = {gvar_name: tf.get_default_graph().get_operation_by_name(gvar_name + "/Assign")
                  for gvar_name in gvar_names}
    init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
    feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
    tf.get_default_session().run(assign_ops, feed_dict=feed_dict)  
  
  #снова получить начальный график v3. На этот раз воспользуемся trainingзаполнителем
  
X = tf.placeholder(tf.float32, shape=[None, height, width, channels], name="X")
training = tf.placeholder_with_default(False, shape=[])
with slim.arg_scope(inception.inception_v3_arg_scope()):
    logits, end_points = inception.inception_v3(X, num_classes=1001, is_training=training)

inception_saver = tf.train.Saver()


#изучить входные данные выходного уровня:
# logits.op.inputs[0].op.inputs[0]  или end_points
#end_points["PreLogits"]

#отказаться от 2-го и 3-го измерений с помощью tf.squeeze() функции
prelogits = tf.squeeze(end_points["PreLogits"], axis=[1, 2])

#добавить окончательный полностью подключенный слой поверх этого слоя:  
n_outputs = len(flower_classes)

with tf.name_scope("new_output_layer"):
    flower_logits = tf.layers.dense(prelogits, n_outputs, name="flower_logits")
    Y_proba = tf.nn.softmax(flower_logits, name="Y_proba")
    

y = tf.placeholder(tf.int32, shape=[None])

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=flower_logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    flower_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="flower_logits") #flower_logits
    training_op = optimizer.minimize(loss, var_list=flower_vars)
    

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(flower_logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver() 
    

    
#Разделите свой набор данных на тренировочный набор и тестовый набор. Обучите модель тренировочному набору и оцените его на тестовом наборе.
#Во-первых, мы хотим представить классы как int, а не строки:
flower_class_ids = {flower_class: index for index, flower_class in enumerate(flower_classes)}


#Будет проще перетасовать набор данных, если мы представим его как список пар filepath / class:
flower_paths_and_classes = []
for flower_class, paths in image_paths.items():
    for path in paths:
        flower_paths_and_classes.append((path, flower_class_ids[flower_class]))
        
#Затем давайте перетасовать набор данных и разделим его на обучающий набор и тестовый набор:


np.random.shuffle(flower_paths_and_classes)
a = flower_paths_and_classes
flower_paths_and_classes_train = a[:int(len(a)*0.6)]
flower_paths_and_classes_valid = a[int(len(a)*0.6):int(len(a)*0.8)]
flower_paths_and_classes_test = a[int(len(a)*0.8):]

n_iterations_per_epoch = len(flower_paths_and_classes_train) // batch_size

print("train:", len(flower_paths_and_classes_train) ,"valid:", len(flower_paths_and_classes_valid), "test", len(flower_paths_and_classes_test))

#flower_paths_and_classes
#print("flower_paths_and_classes", len(flower_paths_and_classes))
#print("flower_paths_and_classes_valid", len(flower_paths_and_classes_valid))
X_train, y_train = prepare_data(flower_paths_and_classes_train)
X_test, y_test = prepare_data(flower_paths_and_classes_test)
X_valid, y_valid = prepare_data(flower_paths_and_classes_valid)        
n_epochs = 10000
#batch_size = 50       

best_loss_val = np.infty
check_interval = 100
checks_since_last_progress = 0
max_checks_without_progress = 20
best_model_params = None 

#step

def shuffle_batch(X, batch_size):
  #rnd_idx = [i for i in range(len(X)) if i>=batch_n*batch_size and i<(batch_n*batch_size+batch_size)]
  #print("rnd_idx", rnd_idx)
  rnd_idx = []
  for i in range(len(X)): rnd_idx.append(i)
  for batch_idx in np.array_split(rnd_idx, batch_size):   # np.array_split делит масим на несколько массивов
      #print("batch_idx", batch_idx)  
      X_batch = X[batch_idx]
      yield X_batch      


 
############################## ВЫБИРАЕМ ПЕРЕМЕННЫЕ ##########################
reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="InceptionV3") # regular expression
restore_saver = tf.train.Saver(reuse_vars) # to restore layers 1-3
#n_batches = len(X_train) // batch_size
#batch_n = 1101
#batch_n1 = 734
#batch_size = 5

h2_cache_2 = np.empty((0, 2048))
h2_cache_valid_2 = np.empty((0, 2048))
#h2_cache = []
#h2_cache_1 = []
#h2_cache_valid_1 = []
#h2_cache_valid = []
################################################################################
with tf.Session() as sess:
    init.run()
    inception_saver.restore(sess, os.path.join(MODELS_PATH, MODEL_NAME))
    
    #for batch_n in range(len(X_train)//batch_size): 
    for X_batch in shuffle_batch(X_train, batch_size):
            h2_cache_1 = sess.run(prelogits, feed_dict={X: X_batch, training: False})
            h2_cache_2 = np.concatenate((h2_cache_2, h2_cache_1))
    #for batch_n in range(len(X_train)//batch_size):
    for X_val in shuffle_batch(X_valid, batch_size):
            h2_cache_valid_1 = sess.run(prelogits, feed_dict={X: X_val, training: False})
            h2_cache_valid_2 = np.concatenate((h2_cache_valid_2, h2_cache_valid_1))

            
############################## СОЗДАЕМ КЭШ ###################################
   
  # h2_cache = prelogits.eval(feed_dict={X: X_train})
   # h2_cache_valid = sess.run(prelogits, feed_dict={X: X_valid})
################################################################################  
    
    

train: 2202 valid: 734 test 734
INFO:tensorflow:Restoring parameters from /content/inception_v3_2016_08_28/inception_v3.ckpt


In [7]:
h2_cache_valid_2.shape

(734, 2048)

In [0]:
h2_cache_without_resize

array([], shape=(0, 2048), dtype=float32)

In [1]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Running setup.py bdist_wheel for gputil ... - done
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.9 GB  | Proc size: 140.7 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [0]:

#h2_cache = np.array(h2_cache)
print(h2_cache[:3])
h2_cache
print(h2_cache.shape)
h2_cache[:3]

[[0.15117478 0.0227656  0.266469   ... 0.21273202 0.20294254 0.        ]
 [0.11786628 0.03779475 0.22354326 ... 0.27310672 0.16936435 0.        ]
 [0.17138407 0.04705916 0.13491091 ... 0.30150726 0.05455118 0.        ]]
(122, 2048)


array([[0.15117478, 0.0227656 , 0.266469  , ..., 0.21273202, 0.20294254,
        0.        ],
       [0.11786628, 0.03779475, 0.22354326, ..., 0.27310672, 0.16936435,
        0.        ],
       [0.17138407, 0.04705916, 0.13491091, ..., 0.30150726, 0.05455118,
        0.        ]], dtype=float32)

In [0]:
np.set_printoptions(threshold=10)
h2_cache[2]


array([0.0046885 , 0.11755972, 0.04769261, ..., 0.15647194, 0.08709227,
       0.00934501], dtype=float32)

In [0]:
# НЕ ТРОГАТЬ
print(h2_cache_2[:3])
h2_cache_2
print(h2_cache_2.shape)

h2_cache_2[:5]

[[0.14001292 0.03021909 0.17526796 ... 0.15923414 0.01626447 0.        ]
 [0.12993246 0.02821892 0.17930259 ... 0.23912324 0.15550253 0.        ]
 [0.01801635 0.07236545 0.05991421 ... 0.00897975 0.00563239 0.00939953]]
(2202, 2048)


array([[0.14001292, 0.03021909, 0.17526796, ..., 0.15923414, 0.01626447,
        0.        ],
       [0.12993246, 0.02821892, 0.17930259, ..., 0.23912324, 0.15550253,
        0.        ],
       [0.01801635, 0.07236545, 0.05991421, ..., 0.00897975, 0.00563239,
        0.00939953],
       [0.06372327, 0.08178102, 0.11291628, ..., 0.0740202 , 0.0315853 ,
        0.        ],
       [0.19616571, 0.0306903 , 0.20578927, ..., 0.23803714, 0.08466715,
        0.        ]])

In [0]:
#shuffled_idx = np.random.permutation(len(X_train))
#np.array_split(h2_cache[6,8,1], 2)
#print(shuffled_idx)
#h2_cache_1 
#print(X_batch)
h2_cache.extend(h2_cache)
h2_cache

In [0]:
prelogits.shape
#y_train.shape

TensorShape([Dimension(None), Dimension(2048)])

In [15]:
tf.reset_default_graph
def shuffle_batch_2(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch
  
batch_size = 200
n_iterations_per_epoch = len(flower_paths_and_classes_train) // 1000
best_loss_val = np.infty
check_interval = 500
checks_since_last_progress = 0
max_checks_without_progress = 100

#X_test_batches, y_test_batches = shuffle_batch(X_test, y_test, n_valid_batches)
#X_valid_batches, y_valid_batches = shuffle_batch(X_valid, y_valid, n_valid_batches)  


with tf.Session() as sess:
  
  
    n_batches = len(X_train) // batch_size

    inception_saver.restore(sess, os.path.join(MODELS_PATH, MODEL_NAME))
    init.run()
    for epoch in range(n_epochs):
        shuffled_idx = np.random.permutation(len(X_train))
        hidden2_batches = np.array_split(h2_cache_2[shuffled_idx], n_batches)
        y_batches = np.array_split(y_train[shuffled_idx], n_batches)
        for hidden2_batch, y_batch in zip(hidden2_batches, y_batches):
            sess.run(training_op, feed_dict={prelogits:hidden2_batch, y:y_batch, training: True})

        accuracy_val = accuracy.eval(feed_dict={prelogits: h2_cache_valid_2, # not shown
                                                y: y_valid})   
        if epoch % 100 == 0:
           print(epoch, "Validation accuracy:", accuracy_val)
  
  
  
  
  
  
    ''' init.run()
    X_test_batches, y_test_batches = shuffle_batch(X_test, y_test, n_valid_batches)
    print(X_test_batches.shape)
    X_valid_batches, y_valid_batches = shuffle_batch(X_valid, y_valid, n_valid_batches)
    inception_saver.restore(sess, os.path.join(MODELS_PATH, MODEL_NAME))
    n_valid_batches = 50
    n_test_batches = 50
    n_epochs = 1000
    for epoch in range(n_epochs):
        X_test_batches, y_test_batches = shuffle_batch(X_test, y_test, n_valid_batches)
        X_valid_batches, y_valid_batches = shuffle_batch(X_valid, y_valid, n_valid_batches)
        huffled_idx = np.random.permutation(len(X_train))
        for iteration in range(len(X_train) // batch_size):
          hidden2_batches = np.array_split(h2_cache_2[shuffled_idx], n_batches)
          y_batches = np.array_split(y_train[shuffled_idx], n_batches)
          sess.run(training_op, feed_dict={prelogits:hidden2_batch, y:y_batch, training: True})
          if iteration % check_interval == 0:
                loss_val = loss.eval(feed_dict={X: X_valid_batches,
                                                y: y_valid_batches})
                if loss_val < best_loss_val:
                    best_loss_val = loss_val
                    checks_since_last_progress = 0
                    best_model_params = get_model_params()
                else:
                    checks_since_last_progress += 1               
        acc_train = accuracy.eval(feed_dict={X: X_test_batches, y: y_test_batches})
        acc_val = accuracy.eval(feed_dict={X: X_valid_batches, y: y_valid_batches})

        print("Epoch {}, train accuracy: {:.4f}%, valid. accuracy: {:.4f}%, valid. best loss: {:.6f}".format(
                  epoch, acc_train * 100, acc_val * 100, best_loss_val))
        if checks_since_last_progress > max_checks_without_progress:
            print("Early stopping!")
            break

    if best_model_params:
        restore_model_params(best_model_params)
    X_valid_batches, y_valid_batches = shuffle_batch(X_valid, y_valid, n_valid_batches)
    acc_test = accuracy.eval(feed_dict={X: X_valid_batches,
                                        y: y_valid_batches})
    print("Final accuracy on test set:", acc_test)
    save_path = saver.save(sess, "./my_mnist_model") 
            
    '''


INFO:tensorflow:Restoring parameters from /content/inception_v3_2016_08_28/inception_v3.ckpt
0 Validation accuracy: 0.24795641
100 Validation accuracy: 0.5217984
200 Validation accuracy: 0.5449591
300 Validation accuracy: 0.5640327
400 Validation accuracy: 0.5708447
500 Validation accuracy: 0.5735695
600 Validation accuracy: 0.56948227
700 Validation accuracy: 0.5776567
800 Validation accuracy: 0.58446866
900 Validation accuracy: 0.5885559
1000 Validation accuracy: 0.59536785
1100 Validation accuracy: 0.58038145
1200 Validation accuracy: 0.5776567
1300 Validation accuracy: 0.5817439
1400 Validation accuracy: 0.58583105
1500 Validation accuracy: 0.57901907
1600 Validation accuracy: 0.58446866
1700 Validation accuracy: 0.58446866
1800 Validation accuracy: 0.5831063
1900 Validation accuracy: 0.57493186
2000 Validation accuracy: 0.58583105
2100 Validation accuracy: 0.5776567
2200 Validation accuracy: 0.58446866
2300 Validation accuracy: 0.5776567
2400 Validation accuracy: 0.5708447
2500 Va

In [0]:
hidden2_batches

<generator object shuffle_batch at 0x7fd0fb1c4308>

In [0]:
len(X_valid)


In [0]:
len(X_valid)

734